#### Import

In [46]:
import pandas as pd
import numpy as np
import numba as nb
import matplotlib.pyplot as plt

#### Functions

In [47]:
@nb.njit(nb.float64[:](nb.int64,
                       nb.int64,
                       nb.float64,
                       nb.float64))
def get_95quantile_maxHn(sim, M, c0, d0):
    sample = np.zeros(sim)
    space = np.linspace(0, 1, M)
    for s in range(sim):
        W = np.cumsum(np.random.randn(M)) / np.sqrt(M)
        B = W - space * W[-1]
        H = B / np.sqrt(space * (1 - space))
        restrH = H[((space >= c0) * (space <= d0)).astype('bool')]
        sample[s] = np.max(restrH)
    return sample


@nb.njit(nb.float64(nb.float64))
def norm_phi_point(x):
    return np.exp(-x**2 / 2) / np.sqrt(2*np.pi)


@nb.njit(nb.float64[:](nb.float64[:]))
def norm_phi_array(x):
    return np.exp(-x**2 / 2) / np.sqrt(2*np.pi)


@nb.njit(nb.float64(nb.float64[:]))
def bw_nrd0(data):
    if len(data) < 2:
        raise ValueError("Need at least 2 data points")
    
    hi = np.std(data)
    lo = min(hi, (np.percentile(data, 75) - np.percentile(data, 25)) / 1.34) if hi else hi or abs(data[0]) or 1.0
    
    return 0.9 * lo * len(data)**(-0.2)


@nb.njit(nb.float64[:](nb.float64[:],
                       nb.int64,
                       nb.float64[:],
                       nb.int64))
def get_KDE_array(data, n, eval_points, m):
    results = np.zeros(m)
    h = bw_nrd0(data)
    
    for j in range(m):
        arg = eval_points[j]
        results[j] = np.mean(norm_phi_array((arg - data) / h)) / h
    return results


@nb.njit(nb.float64(nb.float64[:],
                    nb.int64,
                    nb.float64))
def get_KDE_point(data, n, eval_point):
    h = bw_nrd0(data)
    return np.mean(norm_phi_array((eval_point - data) / h)) / h


@nb.njit(nb.float64(nb.int64, nb.float64[:], nb.float64))
def H(tau, y, q):
    n = len(y)
    yL = y[:tau]
    yR = y[tau:]
    
    muL = np.quantile(yL, q)
    muR = np.quantile(yR, q)
    
    fL = get_KDE_point(yL, tau, muL)
    fR = get_KDE_point(yR, n - tau, muR)
    
    kappaLsq = q * (1 - q) / (fL**2)
    kappaRsq = q * (1 - q) / (fR**2)
    
    return (muL - muR) / (kappaLsq / tau + kappaRsq / (n - tau)) ** (1 / 2)


@nb.njit(nb.float64(nb.float64[:], 
                    nb.float64,
                    nb.float64,
                    nb.int64))
def include_ukraine(y, ukr, q, cutoff):
    logukr = np.log(ukr)
    
    yy = np.append(y, logukr)
    n = len(yy)
    
    maxH = -np.inf
    
    for tau in range(cutoff, n - cutoff):
        currentH = H(tau, yy, q)
        if currentH >= maxH:
            maxH = currentH
    return maxH

#### Data

In [48]:
df = pd.read_csv('../cow_ssba.csv')
x = df['start_dates'].to_numpy().astype('float')
z = df['battle_deaths'].to_numpy().astype('float')
y = np.log(z)
n = len(x)

#### Parameters

In [49]:
q = .75
cutoff = 5

#### Simulations

In [85]:
ukraine_sizes = np.arange(1000, 50_000, 100)
mukr = len(ukraine_sizes)
maxHarray = np.zeros(mukr)

for j in range(mukr):
    maxHarray[j] = include_ukraine(y, ukraine_sizes[j], q, cutoff)

In [86]:
plt.plot(ukraine_sizes, maxHarray)
plt.ylabel('Hn plot')
plt.xlabel('hypothetical Russo-Ukrainan casualties')
plt.show()

In [33]:
np.arange(7, 14, 3)

array([ 7, 10, 13])

In [60]:
ukraine_sizes = np.arange(1000, 1_000_000, 1000)
ukraine_sizes

array([  1000,   2000,   3000,   4000,   5000,   6000,   7000,   8000,
         9000,  10000,  11000,  12000,  13000,  14000,  15000,  16000,
        17000,  18000,  19000,  20000,  21000,  22000,  23000,  24000,
        25000,  26000,  27000,  28000,  29000,  30000,  31000,  32000,
        33000,  34000,  35000,  36000,  37000,  38000,  39000,  40000,
        41000,  42000,  43000,  44000,  45000,  46000,  47000,  48000,
        49000,  50000,  51000,  52000,  53000,  54000,  55000,  56000,
        57000,  58000,  59000,  60000,  61000,  62000,  63000,  64000,
        65000,  66000,  67000,  68000,  69000,  70000,  71000,  72000,
        73000,  74000,  75000,  76000,  77000,  78000,  79000,  80000,
        81000,  82000,  83000,  84000,  85000,  86000,  87000,  88000,
        89000,  90000,  91000,  92000,  93000,  94000,  95000,  96000,
        97000,  98000,  99000, 100000, 101000, 102000, 103000, 104000,
       105000, 106000, 107000, 108000, 109000, 110000, 111000, 112000,
      

In [18]:
a = np.random.random(18)
a[:5]

array([0.04003525, 0.05426294, 0.17775087, 0.84362941, 0.29071039])

In [19]:
len(a[:5])

5

In [84]:
alphahat = .499
muhat = 10940
thetahat = 1.022

bhat = muhat
ahat = thetahat
phat = alphahat

bhat * (-1 + 2**(1 / phat)) ** (-1 / ahat)

3720.432469714857